# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [4]:
df = pd.read_csv('AviationData.csv', encoding='latin1')

/var/folders/fv/82cs2c0n4p7fnj4k3xl34p1m0000gn/T/ipykernel_19339/2656823905.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('AviationData.csv', encoding='latin1')


In [5]:
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [6]:
print(df['Aircraft.Category'].value_counts(dropna=False))

print(df['Amateur.Built'].value_counts(dropna=False))

print(df['Event.Date'].head())

Aircraft.Category
NaN                  56602
Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: count, dtype: int64
Amateur.Built
No     80312
Yes     8475
NaN      102
Name: count, dtype: int64
0    1948-10-24
1    1962-07-19
2    1974-08-30
3    1977-06-19
4    1979-08-02
Name: Event.Date, dtype: object


In [8]:
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')

df = df[df['Event.Date'].dt.year >= 1983]

print(df['Event.Date'].head())

3600   1983-01-01
3601   1983-01-01
3602   1983-01-01
3603   1983-01-01
3604   1983-01-01
Name: Event.Date, dtype: datetime64[ns]


In [9]:
df = df[df['Amateur.Built'] == 'No']

print(df['Amateur.Built'].value_counts(dropna=False))

Amateur.Built
No    76960
Name: count, dtype: int64


In [10]:
df = df[df['Aircraft.Category'].str.lower() == 'airplane']

print(df['Aircraft.Category'].value_counts(dropna=False))

Aircraft.Category
Airplane    21447
Name: count, dtype: int64


### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [12]:
# Assumption: If a value is missing, assume 0
# Fill missing injury counts with 0
df[['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']] = df[['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']].fillna(0)

# Construct Total_Occupants
df['Total_Occupants'] = df['Total.Fatal.Injuries'] + df['Total.Serious.Injuries'] + df['Total.Minor.Injuries'] + df['Total.Uninjured']

df[['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured', 'Total_Occupants']].head()


,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Total_Occupants
4149,0.0,0.0,0.0,588.0,588.0
4150,0.0,0.0,0.0,588.0,588.0
4171,1.0,1.0,0.0,0.0,2.0
4285,1.0,0.0,0.0,4.0,5.0
5957,0.0,0.0,0.0,289.0,289.0


In [ ]:
# Assumption: The probability of fatal or serious injury on a flight can be approximated as the fraction of occupants in that category.

# Fraction of occupants who were seriously injured
df['Fatality_Rate'] = df['Total.Fatal.Injuries'] / df['Total_Occupants']

# Fraction of occupants who were seriously injured
df['Serious_Injury_Rate'] = df['Total.Serious.Injuries'] / df['Total_Occupants']

# Combine fata + serious injury for one safety metric
df['Fatal_or_Serious_Rate'] = df['Fatality_Rate'] + df['Serious_Injury_Rate']

df[['Total_Occupants', 'Fatality_Rate', 'Serious_Injury_Rate', 'Fatal_or_Serious_Rate']].head()

# Fraction-based approach accounts for varying flight sizes

,Total_Occupants,Fatality_Rate,Serious_Injury_Rate,Fatal_or_Serious_Rate
4149,588.0,0.0,0.0,0.0
4150,588.0,0.0,0.0,0.0
4171,2.0,0.5,0.5,1.0
4285,5.0,0.2,0.0,0.2
5957,289.0,0.0,0.0,0.0


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [17]:
print(df['Aircraft.damage'].value_counts(dropna=False))


Aircraft.damage
Substantial    16990
Destroyed       2316
NaN             1227
Minor            817
Unknown           97
Name: count, dtype: int64


In [18]:
# Standardize and clean
df['Aircraft.damage'] = df['Aircraft.damage'].str.lower().str.strip()

# Replace missing values with "unknown"
df['Aircraft.damage'] = df['Aircraft.damage'].fillna('unknown')
df['Aircraft.damage'] = df['Aircraft.damage'].replace('', 'unknown')

# Check cleaned counts

print(df['Aircraft.damage'].value_counts())

Aircraft.damage
substantial    16990
destroyed       2316
unknown         1324
minor            817
Name: count, dtype: int64


In [19]:
# 1 = Destroyed, 0 = Not Destroyed
df['Total_Destruction'] = df['Aircraft.damage'].apply(lambda x: 1 if x == 'destroyed' else 0)

print(df['Total_Destruction'].value_counts())

Total_Destruction
0    19131
1     2316
Name: count, dtype: int64


In [20]:
# Cross-check between injuries and destruction
df.groupby('Total_Destruction')[['Fatality_Rate', 'Serious_Injury_Rate']].mean()

,Fatality_Rate,Serious_Injury_Rate
Total_Destruction,,
0,0.096109,0.119048
1,0.788422,0.050039


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [21]:
# Count makes
print('Number of unique makes:', df['Make'].nunique())

# Most common makes
print(df['Make'].value_counts().head(20))

# Missing values
print('Missing values in Make:', df['Make'].isna().sum())

Number of unique makes: 1332
Make
CESSNA                4867
PIPER                 2803
Cessna                2279
Piper                 1186
BOEING                1037
BEECH                 1018
Beech                  413
MOONEY                 238
Boeing                 227
CIRRUS DESIGN CORP     218
AIR TRACTOR INC        217
AIRBUS                 215
BELLANCA               158
AERONCA                149
MAULE                  144
Mooney                 125
EMBRAER                123
Air Tractor            117
LUSCOMBE                95
DEHAVILLAND             91
Name: count, dtype: int64
Missing values in Make: 3


## Cleaning Tasks for Make Column
- Standardize capitalization
- Remove whitespace
- Handle missing values
- Standardize manufacturer names
- Filter out makes with few occurances

In [27]:
# Standardize capitalization
df['Make'] = df['Make'].str.title()

# Strip whitespace
df['Make'] = df['Make'].str.strip()

# Drop missing value rows
df = df.dropna(subset=['Make'])

# Name variations
df['Make'] = df['Make'].replace({
  'Air Tractor Inc': 'Air Tractor',
  'Cirrus Design Corp': 'Cirrus',
  'Dehavilland': 'De Havilland',
  'Mcdonnell-Douglas': 'Mcdonnell Douglas'
})

# Filter few accidents out
make_counts = df['Make'].value_counts()

makes_to_keep = make_counts[make_counts >= 50].index
df = df[df['Make'].isin(makes_to_keep)]

# Check
print(df['Make'].value_counts().head(20))
print('Number of unique makes after cleaning:', df['Make'].nunique())

Make
Cessna               7146
Piper                3989
Beech                1431
Boeing               1264
Air Tractor           425
Mooney                363
Cirrus                357
Airbus                243
Bellanca              219
Maule                 215
Aeronca               200
De Havilland          168
Champion              158
Embraer               153
Grumman               147
Luscombe              141
Stinson               129
Mcdonnell Douglas     108
North American        106
Taylorcraft            93
Name: count, dtype: int64
Number of unique makes after cleaning: 33


### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [28]:
print('Missing values in Model:', df['Model'].isna().sum())

print(df['Model'].value_counts().head(20))

Missing values in Model: 13
Model
172          769
737          403
152          316
182          304
172S         276
PA28         273
172N         249
SR22         240
180          213
A36          181
172M         180
150          179
PA-18-150    175
PA-28-140    169
172P         143
140          117
172R         109
170B         107
PA-28-180    105
PA-28-161    102
Name: count, dtype: int64


In [29]:
# Drop rows with missing Model
df = df.dropna(subset=['Model'])

# Standardize text
df['Model'] = df['Model'].str.upper().str.strip()

In [32]:
# Create make/model identifier
df['Make_Model'] = df['Make'] + " " + df['Model']

print(df[['Make', 'Model', 'Make_Model']].head())
print('Number of unique Make_Model combinations:', df['Make_Model'].nunique())

              Make      Model          Make_Model
4150        Boeing        747          Boeing 747
4171         Piper  PA-28-140     Piper PA-28-140
4285  De Havilland      DHC-6  De Havilland DHC-6
6760        Boeing    727-200      Boeing 727-200
6806         Beech        C35           Beech C35
Number of unique Make_Model combinations: 2115


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [33]:
# Engine Type
print(df['Engine.Type'].value_counts(dropna=False).head(20))

Engine.Type
Reciprocating      12835
NaN                 3214
Turbo Prop           931
Turbo Fan            701
Unknown              105
Turbo Jet             71
Geared Turbofan       12
Turbo Shaft            9
UNK                    1
Name: count, dtype: int64


In [ ]:
df['Engine.Type'] = df['Engine.Type'].str.upper().str.strip()
df['Engine.Type'] = df['Engine.Type'].replace('', 'Unknown')

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized